# Generate some data

In [1]:
from bayes_window.generative_models import generate_fake_spikes
from bayes_window.visualization import fake_spikes_explore, plot_data_and_posterior
from bayes_window import models
from bayes_window.fitting import fit_numpyro
from bayes_window.utils import add_data_to_posterior

In [2]:

df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=10,
                                                                n_neurons=7,
                                                                n_mice=6,
                                                                dur=7,
                                                               mouse_response_slope=16)

# Exploratory plot without any fitting

In [3]:

from bayes_window.visualization import fake_spikes_explore
charts=fake_spikes_explore(df,df_monster,index_cols)
[chart.display() for chart in charts];
#fig_mice, fig_select, fig_neurons, fig_trials, fig_isi + fig_overlay, bar, box, fig_raster, bar_combined

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.LayerChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.LayerChart(...)

# Estimate with mouse>neuron

TODO 
- why no line in bayes?

In [4]:
df_monster['neuron']

1         0
2         0
3         0
4         0
5         0
         ..
169793    6
169794    6
169795    6
169796    6
169797    6
Name: neuron, Length: 168958, dtype: object

In [5]:
from bayes_window import workflow
from importlib import reload
reload(workflow)

workflow.estimate_slope(df,y='isi', levels=('stim', 'mouse', 'neuron'), hold_for_facet=False,add_box=True)

n(Divergences) = 1


(alt.LayerChart(...),
       mouse neuron  i_trial  mouse_code  neuron_code  log_firing_rate  \
 0   m0bayes      0      0.0         0.0          0.0         0.052316   
 1   m0bayes      1      0.0         0.0          0.0        -0.004012   
 2   m0bayes      2      0.0         0.0          0.0        -0.123396   
 3   m0bayes      3      0.0         0.0          0.0        -0.195208   
 4   m0bayes      4      0.0         0.0          0.0        -0.134848   
 5   m0bayes      5      0.0         0.0          0.0        -0.007653   
 6   m0bayes      6      0.0         0.0          0.0        -0.007089   
 7   m1bayes      0      0.0         0.0          0.0         0.198516   
 8   m1bayes      1      0.0         0.0          0.0         0.467445   
 9   m1bayes      2      0.0         0.0          0.0         0.281672   
 10  m1bayes      3      0.0         0.0          0.0         0.394367   
 11  m1bayes      4      0.0         0.0          0.0         0.090654   
 12  m1bayes    

In [6]:
#Q
from bayes_window import workflow
from importlib import reload
reload(workflow)

workflow.estimate_slope(df,y='isi', levels=('stim', 'mouse', 'neuron'), hold_for_facet=False,
                        add_box=False,
                        add_data=False)

n(Divergences) = 1


(alt.LayerChart(...),
    i_trial  mouse_code  neuron_code  mouse  stim  neuron neuron_x_mouse  \
 0        0           0            0      0     0       0       0m0bayes   
 1        0           0            6      0     0       1       1m0bayes   
 2        0           0           12      0     0       2       2m0bayes   
 3        0           0           18      0     0       3       3m0bayes   
 4        0           0           24      0     0       4       4m0bayes   
 5        0           0           30      0     0       5       5m0bayes   
 6        0           0           36      0     0       6       6m0bayes   
 
    log_firing_rate  Firing rate  firing_rate  i_spike       isi  \
 0         1.822148    66.396887    66.396887     44.5  0.079007   
 1         1.768545    58.687389    58.687389     49.5  0.070500   
 2         2.160362   144.664559   144.664559     63.5  0.055232   
 3         2.170949   148.234472   148.234472     78.5  0.043557   
 4         2.481666   303.15

With box plot:

In [7]:
reload(workflow)
workflow.estimate_slope(df,y='isi', levels=('stim', 'mouse', 'neuron'), hold_for_facet=False,add_box=True)


n(Divergences) = 1


(alt.LayerChart(...),
     mouse  neuron  i_trial  mouse_code  neuron_code  log_firing_rate  \
 0     0.0     0.0      0.0         0.0          0.0         0.052316   
 1     0.0     1.0      0.0         0.0          0.0        -0.004012   
 2     0.0     2.0      0.0         0.0          0.0        -0.123396   
 3     0.0     3.0      0.0         0.0          0.0        -0.195208   
 4     0.0     4.0      0.0         0.0          0.0        -0.134848   
 5     0.0     5.0      0.0         0.0          0.0        -0.007653   
 6     0.0     6.0      0.0         0.0          0.0        -0.007089   
 7     1.0     0.0      0.0         0.0          0.0         0.198516   
 8     1.0     1.0      0.0         0.0          0.0         0.467445   
 9     1.0     2.0      0.0         0.0          0.0         0.281672   
 10    1.0     3.0      0.0         0.0          0.0         0.394367   
 11    1.0     4.0      0.0         0.0          0.0         0.090654   
 12    1.0     5.0      0.0  

In [8]:
workflow.estimate_slope(df,y='isi', levels=('stim', 'mouse', 'neuron'), hold_for_facet=True,add_box=False)


n(Divergences) = 1


(alt.LayerChart(...),
     mouse  neuron  i_trial  mouse_code  neuron_code  log_firing_rate  \
 0     0.0     0.0      0.0         0.0          0.0         0.052316   
 1     0.0     1.0      0.0         0.0          0.0        -0.004012   
 2     0.0     2.0      0.0         0.0          0.0        -0.123396   
 3     0.0     3.0      0.0         0.0          0.0        -0.195208   
 4     0.0     4.0      0.0         0.0          0.0        -0.134848   
 5     0.0     5.0      0.0         0.0          0.0        -0.007653   
 6     0.0     6.0      0.0         0.0          0.0        -0.007089   
 7     1.0     0.0      0.0         0.0          0.0         0.198516   
 8     1.0     1.0      0.0         0.0          0.0         0.467445   
 9     1.0     2.0      0.0         0.0          0.0         0.281672   
 10    1.0     3.0      0.0         0.0          0.0         0.394367   
 11    1.0     4.0      0.0         0.0          0.0         0.090654   
 12    1.0     5.0      0.0  

# Estimate with neuron>mouse

In [9]:

workflow.estimate_slope(df,y='isi', levels=('stim', 'neuron','mouse'), hold_for_facet=False)

n(Divergences) = 0


(alt.LayerChart(...),
     neuron  mouse  i_trial  mouse_code  neuron_code  log_firing_rate  \
 0      0.0    0.0      0.0         0.0          0.0         0.052316   
 1      0.0    1.0      0.0         0.0          0.0         0.198516   
 2      0.0    2.0      0.0         0.0          0.0         0.287406   
 3      0.0    3.0      0.0         0.0          0.0         0.675458   
 4      0.0    4.0      0.0         0.0          0.0         0.719302   
 5      0.0    5.0      0.0         0.0          0.0         1.084922   
 6      1.0    0.0      0.0         0.0          0.0        -0.004012   
 7      1.0    1.0      0.0         0.0          0.0         0.467445   
 8      1.0    2.0      0.0         0.0          0.0         0.207119   
 9      1.0    3.0      0.0         0.0          0.0         0.825280   
 10     1.0    4.0      0.0         0.0          0.0         0.868403   
 11     1.0    5.0      0.0         0.0          0.0         0.741480   
 12     2.0    0.0      0.0  

In [10]:
from bayes_window import workflow, models
reload(workflow)


<module 'bayes_window.workflow' from '/home/m/mmy/bayes-window/bayes_window/workflow.py'>

In [11]:
cposter,df_both=workflow.estimate_posteriors(df,y='log_firing_rate',
                                             model=models.model_single_normal,
                                             add_data=False,
                                            )

  0%|          | 0/1100 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import altair as alt
# mean firing rate per trial per mouse
width = 50
fig_trials = alt.Chart().mark_line(fill=None, ).encode(
    x=alt.X('stim'),
    y=alt.Y('log_firing_rate', scale=alt.Scale(zero=False)),
    color='neuron:N',
    detail='i_trial:Q',
    opacity=alt.value(1),
    size=alt.value(.9),
).properties(
    # columns=5,
    width=width,
    height=300
)#.facet(row='mouse',column='neuron').properties(title='All trials and neurons',)
#fig_trials.facet(row='mouse',column='neuron')

In [ ]:
alt.layer(fig_trials,data=df_both).facet(row='mouse',column='neuron')

In [ ]:
alt.layer(cposter,data=df_both).facet(row='mouse',column='neuron')

In [ ]:
alt.layer(cposter,fig_trials,data=df_both)#.facet(row='mouse',column='neuron')

In [ ]:
alt.layer(cposter, fig_trials,data=df_both).facet(row='mouse',column='neuron')